In [ ]:
#| default_exp handlers.maris_dump

# MARIS dump
> Data pipeline (handler) to convert global MARIS db dump into `NetCDF` format

The input data is a dump from already imported MARIS datasets.


**Questions**:
1. is `decayedto` used and in what context? (question to be answered for all columns actually)
2. what are units of uncertainty?
3. ref_id vs. zoterourl vs. displaytext
   1.  e.g ref_id=129 and 130 points to the same dataset?

**Dev. board**: https://trello.com/b/IszgV1bj/marisco

## Packages import

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import pandas as pd
from tqdm import tqdm
from functools import partial
import fastcore.all as fc

from pathlib import Path

from marisco.callbacks import (Callback, Transformer, SanitizeLonLatCB)
from marisco.metadata import (GlobAttrsFeeder, BboxCB,
                              DepthRangeCB, TimeRangeCB,
                              ZoteroCB, KeyValuePairCB)
from marisco.configs import lut_path, cdl_cfg, cfg, nc_tpl_path
from marisco.serializers import NetCDFEncoder

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fname_in = Path().home() / 'pro/data/maris/all-maris.txt'
dir_dest = '../../_data/output/dump'
# fname_out = '../../_data/output/helcom.nc'

## Utils

In [ ]:
# | export
def load_dump(fname): 
    return pd.read_csv(fname, sep='\t', encoding='ISO-8859-1')   

def load_data(df:pd.DataFrame, # MARIS global dump 
                 ref_id:int, # Reference id of interest
                 ):
    "Load specific MARIS dataset through its ref_id."
    lut = {
        'Sediment': 'sediment',
        'Seawater': 'seawater',
        'Suspended matter': 'suspended-matter',
        'Biota': 'biota'}
    dfs = {}
    for name, grp in df[df.ref_id  == ref_id].groupby('samptype'):
        dfs[lut[name]] = grp
    return dfs

def get_zotero_key(dfs):
    return dfs[next(iter(dfs))][['zoterourl']].iloc[0].values[0].split('/')[-1]

def get_fname(dfs):
    id, name = dfs[next(iter(dfs))][['ref_id', 'displaytext']].iloc[0]
    name = name.replace(',', '').replace('.', '').replace('-', ' ').split(' ')
    return '-'.join(([str(id)] + name)) + '.nc'

## Load data

In [ ]:
df = load_dump(fname_in)

In [ ]:
# 52
ref_id = 52

## Data transformation pipeline

### Normalize nuclide names

In [ ]:
#| export
def get_varnames_lut():
    fname = lut_path() / 'dbo_nuclide.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['nuclide_id', 'nc_name'])
    return df_nuclide.set_index('nuclide_id').to_dict()['nc_name']

In [ ]:
# | export
class RemapRdnNameCB(Callback):
    "Remap to MARIS radionuclide names."
    def __init__(self,
                 fn_lut=get_varnames_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['nuclide_id'] = tfm.dfs[k]['nuclide_id'].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[RemapRdnNameCB()])

print(tfm()['biota']['nuclide_id'].unique())

['sr90' 'cs137' 'pu238' 'pu239_240_tot']


### Rename columns

In [ ]:
dfs['biota'].columns

Index(['sample_id', 'area_id', 'areaname', 'samptype_id', 'samptype', 'ref_id',
       'displaytext', 'zoterourl', 'ref_note', 'datbase', 'lab_id', 'lab',
       'latitude', 'longitude', 'begperiod', 'endperiod', 'samplingyear',
       'totdepth', 'sampdepth', 'station', 'samplabcode', 'species_id',
       'taxonname', 'taxonrank', 'biogroup', 'biogroup_id', 'taxondb',
       'taxondbid', 'taxondburl', 'taxonrepname', 'bodypar_id', 'bodypar',
       'sliceup', 'slicedown', 'sedtype_id', 'sedtype', 'sedrepname',
       'nuclide_id', 'nusymbol', 'volume', 'salinity', 'temperatur',
       'filtered', 'filtpore', 'samparea', 'drywt', 'wetwt', 'percentwt',
       'sampmet_id', 'sampmet', 'prepmet_id', 'prepmet', 'drymet_id', 'drymet',
       'counmet_id', 'counmet', 'decayedto', 'detection', 'activity',
       'uncertaint', 'unit_id', 'unit', 'vartype', 'freq', 'rangelow',
       'rangeupp', 'profile', 'transect_id', 'measure_note', 'shapetype_id',
       'profile_id', 'sampnote', 'ref_full

In [ ]:
#| export
# To be added: endperiod, totdepth, lab
def renaming_rules():
    vars = cdl_cfg()['vars']
    # Define column names renaming rules
    return {
        'latitude': vars['defaults']['lat']['name'],
        'longitude': vars['defaults']['lon']['name'],
        'begperiod': vars['defaults']['time']['name'],
        'sampdepth': vars['defaults']['depth']['name'],
        'uncertaint': vars['suffixes']['uncertainty']['name'],
        'unit_id': vars['suffixes']['unit']['name'],
        'detection': vars['suffixes']['detection_limit']['name'],
        'area_id': vars['defaults']['area']['name'], 
        'species_id': vars['bio']['species']['name'],
        'biogroup_id': vars['bio']['bio_group']['name'],
        'bodypar_id': vars['bio']['body_part']['name'],
        'sedtype_id': vars['sed']['sed_type']['name'],
        'volume': vars['suffixes']['volume']['name'],
        'salinity': vars['suffixes']['salinity']['name'],
        'temperatur': vars['suffixes']['temperature']['name'],
        'sampmet_id': vars['suffixes']['sampling_method']['name'],
        'prepmet_id': vars['suffixes']['preparation_method']['name'],
        'counmet_id': vars['suffixes']['counting_method']['name'],
        'activity': 'value',
        'nuclide_id': 'nuclide'
    }

In [ ]:
#| export
class RenameColumnCB(Callback):
    "Renaming variables to MARIS standard names."
    def __init__(self,
                 renaming_rules=renaming_rules):
        fc.store_attr()

    def __call__(self, tfm):
        lut = renaming_rules()
        coi = lut.keys()
        for k in tfm.dfs.keys():
            # Select cols of interest
            tfm.dfs[k] = tfm.dfs[k].loc[:, coi]
            # Rename cols
            tfm.dfs[k].rename(columns=lut, inplace=True)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB()
    ])

print(tfm()['biota'])

              lat         lon     time  depth  unc  unit  dl  area  species  \
757776  42.316667  132.316667  00:00.0    NaN  NaN     5   <  4307      712   
757777  42.316667  132.316667  00:00.0    NaN  NaN     5  DE  4307      712   
757778  42.316667  132.316667  00:00.0    NaN  NaN     5  DE  4307      712   
757779  42.316667  132.316667  00:00.0    NaN  NaN     5  DE  4307      712   
757780  42.316667  132.316667  00:00.0    NaN  NaN     5   <  4307      695   
757781  42.316667  132.316667  00:00.0    NaN  NaN     4  DE  4307      695   

        bio_group  body_part  sed_type  vol  sal  temp  sampmet  prepmet  \
757776          4          0         0  NaN  NaN   NaN        0        0   
757777          4          0         0  NaN  NaN   NaN        0        0   
757778          4          0         0  NaN  NaN   NaN        0        0   
757779          4          0         0  NaN  NaN   NaN        0        0   
757780          8          0         0  NaN  NaN   NaN        0   

### Drop NaN only columns

In [ ]:
#| export
class DropNAColumnsCB(Callback):
    "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables)."
    def __init__(self, na_value=0):
        fc.store_attr()
        
    def isMarisNA(self, col): 
        return len(col.unique()) == 1 and col.iloc[0] == self.na_value
    
    def dropMarisNA(self, df):
        na_cols = [col for col in df.columns if self.isMarisNA(df[col])]
        return df.drop(labels=na_cols, axis=1)
        
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = tfm.dfs[k].dropna(axis=1, how='all')
            tfm.dfs[k] = self.dropMarisNA(tfm.dfs[k])

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB()
    ])

print(tfm()['biota'])

              lat         lon     time  unit  dl  area  species  bio_group  \
757776  42.316667  132.316667  00:00.0     5   <  4307      712          4   
757777  42.316667  132.316667  00:00.0     5  DE  4307      712          4   
757778  42.316667  132.316667  00:00.0     5  DE  4307      712          4   
757779  42.316667  132.316667  00:00.0     5  DE  4307      712          4   
757780  42.316667  132.316667  00:00.0     5   <  4307      695          8   
757781  42.316667  132.316667  00:00.0     4  DE  4307      695          8   

           value        nuclide  
757776  0.170000           sr90  
757777  0.142478          cs137  
757778  0.000548          pu238  
757779  0.002389  pu239_240_tot  
757780  0.440000          cs137  
757781  0.176635  pu239_240_tot  


### Sanitize detection limit values

In [ ]:
#| export
def get_dl_lut():
    fname = lut_path() / 'dbo_detectlimit.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['name', 'id'])
    return df_nuclide.set_index('name').to_dict()['id']

In [ ]:
get_dl_lut()

{'Not Available': 0, '=': 1, '<': 2, 'ND': 3, 'DE': 4}

In [ ]:
#| export
class SanitizeDetectionLimitCB(Callback):
    "Assign Detection Limit name to its id based on MARIS nomenclature."
    def __init__(self,
                 fn_lut=get_dl_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['dl'] = tfm.dfs[k]['dl'].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB()
    ])

# print(tfm()['sediment'])
df_test = tfm()['sediment']

In [ ]:
df_test.columns

Index(['lat', 'lon', 'time', 'depth', 'unit', 'dl', 'area', 'value',
       'nuclide'],
      dtype='object')

In [ ]:
df_test.dl.unique()

array([1, 2, 4])

### Reshape: long to wide

In [ ]:
#| export
class ReshapeLongToWide(Callback):
    "Convert data from long to wide with renamed columns."
    def __init__(self, value_col='nuclide'):
        fc.store_attr()
        self.derived_cols = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
    
    def renamed_cols(self, cols):
        return [f'{inner}_{outer}' if inner else outer for outer, inner in cols]

    def pivot(self, df):
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        df.reset_index(names='sample', inplace=True)
        idx = list(set(df.columns) - set([self.value_col] + derived_coi))
        return df.pivot_table(index=idx,
                              columns=self.value_col,
                              values=derived_coi,
                              fill_value=0,
                              aggfunc=lambda x: x
                              ).reset_index()

    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = self.pivot(tfm.dfs[k])
            tfm.dfs[k].columns = self.renamed_cols(tfm.dfs[k].columns)
            # tfm.dfs[k].index.name = 'sample'
            tfm.dfs[k].set_index('sample', inplace=True)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ReshapeLongToWide()
    ])

print(tfm()['sediment'])

         depth        value  area     time        lat         lon  ac228_dl  \
sample                                                                        
755699  3570.0     0.120000  4307  00:00.0  41.916667  134.333333         0   
755702  1512.0     0.300000  4307  00:00.0  42.316667  132.316667         0   
755703  1512.0     2.300000  4307  00:00.0  42.316667  132.316667         0   
755704  1512.0     0.013000  4307  00:00.0  42.316667  132.316667         0   
755705  1512.0     0.760000  4307  00:00.0  42.316667  132.316667         0   
757500  2991.0     0.149666  4307  00:00.0  38.001944  134.951389         0   
757501  2991.0     1.000000  4307  00:00.0  38.001944  134.951389         0   
757502  2991.0     0.018000  4307  00:00.0  38.001944  134.951389         0   
757503  2991.0     0.140000  4307  00:00.0  38.001944  134.951389         0   
757754  3245.0     0.300000  4307  00:00.0  40.795000  134.866667         0   
757755  3245.0     1.400000  4307  00:00.0  40.79500

### Encode time (seconds since ...)

In [ ]:
#| export
class EncodeTimeCB(Callback):
    "Encode time as `int` representing seconds since xxx (TBD)."  
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k].time = 9999

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ReshapeLongToWide(),
    EncodeTimeCB()
    ])

print(tfm()['sediment'])

         depth        value  area  time        lat         lon  ac228_dl  \
sample                                                                     
755699  3570.0     0.120000  4307  9999  41.916667  134.333333         0   
755702  1512.0     0.300000  4307  9999  42.316667  132.316667         0   
755703  1512.0     2.300000  4307  9999  42.316667  132.316667         0   
755704  1512.0     0.013000  4307  9999  42.316667  132.316667         0   
755705  1512.0     0.760000  4307  9999  42.316667  132.316667         0   
757500  2991.0     0.149666  4307  9999  38.001944  134.951389         0   
757501  2991.0     1.000000  4307  9999  38.001944  134.951389         0   
757502  2991.0     0.018000  4307  9999  38.001944  134.951389         0   
757503  2991.0     0.140000  4307  9999  38.001944  134.951389         0   
757754  3245.0     0.300000  4307  9999  40.795000  134.866667         0   
757755  3245.0     1.400000  4307  9999  40.795000  134.866667         0   
757756  3245

### Sanitize coordinates

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(),
    SanitizeLonLatCB()
    ])

# print(tfm()['sediment'])
df_debug = tfm()['sediment']

In [ ]:
# pu239_240_tot_dl

In [ ]:
df_debug.pu239_240_tot_dl.values

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

## Encode to NetCDF

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(),
    SanitizeLonLatCB()
    ])

dfs_tfm = tfm()

In [ ]:
tfm.logs

['Remap to MARIS radionuclide names.',
 'Renaming variables to MARIS standard names.',
 "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables).",
 'Assign Detection Limit name to its id based on MARIS nomenclature.',
 'Convert data from long to wide with renamed columns.',
 'Encode time as `int` representing seconds since xxx (TBD).',
 'Drop row when both longitude & latitude equal 0.']

In [ ]:
#| export
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
#| export
def get_attrs(tfm, zotero_key='26VMZZ2Q', kw=kw):
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(cfg()),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
get_attrs(tfm, zotero_key='3W354SQG', kw=kw)

{'geospatial_lat_min': '134.9513889',
 'geospatial_lat_max': '42.31666667',
 'geospatial_lon_min': '132.0',
 'geospatial_lon_max': '38.00194444',
 'geospatial_bounds': 'POLYGON ((132 38.00194444, 134.9513889 38.00194444, 134.9513889 42.31666667, 132 42.31666667, 132 38.00194444))',
 'geospatial_vertical_max': '0',
 'geospatial_vertical_min': '-3570.0',
 'time_coverage_start': '1970-01-01T02:46:39',
 'time_coverage_end': '1970-01-01T02:46:39',
 'title': 'Radioactivity Monitoring of the Irish Marine Environment 1991 and 1992',
 'summary': '',
 'creator_name': '[{"creatorType": "author", "firstName": "A.", "lastName": "McGarry"}, {"creatorType": "author", "firstName": "S.", "lastName": "Lyons"}, {"creatorType": "author", "firstName": "C.", "lastName": "McEnri"}, {"creatorType": "author", "firstName": "T.", "lastName": "Ryan"}, {"creatorType": "author", "firstName": "M.", "lastName": "O\'Colmain"}, {"creatorType": "author", "firstName": "J.D.", "lastName": "Cunningham"}]',
 'keywords': 'oc

In [ ]:
#| export
def encode(fname_in, fname_out, nc_tpl_path, **kwargs):
    df = load_dump(fname_in)
    ref_ids = kwargs.get('ref_ids', df.ref_id.unique())
    print('Encoding ...')
    # for ref_id in ref_ids:
    for ref_id in tqdm(ref_ids, leave=False):
        dfs = load_data(df, ref_id)
        print(get_fname(dfs))
        # print(get_fname(dfs))    
        tfm = Transformer(dfs, cbs=[
            RemapRdnNameCB(),
            RenameColumnCB(),
            DropNAColumnsCB(),
            SanitizeDetectionLimitCB(),
            ReshapeLongToWide(),
            EncodeTimeCB(),
            SanitizeLonLatCB()
            ])
        
        # species_lut = get_maris_species(fname_in, 'species_helcom.pkl')
        # enums_xtra = {
        #     'species_t': {info['name']: info['id'] 
        #                   for info in species_lut.values() if info['name'] != ''}
        # }
        encoder = NetCDFEncoder(tfm(), 
                                src_fname=nc_tpl_path,
                                dest_fname=Path(fname_out) / get_fname(dfs), 
                                global_attrs=get_attrs(tfm, zotero_key=get_zotero_key(dfs), kw=kw),
                                verbose=kwargs.get('verbose', False)
                                # enums_xtra=enums_xtra
                                )
        encoder.encode()

In [ ]:
encode(fname_in, dir_dest, nc_tpl_path(), verbose=False, ref_ids=[ref_id])

Encoding ...
52-Amano-et-al-1996.nc
Group: biota, Variable: sample
Group: biota, Variable: lon
Group: biota, Variable: lat
Group: biota, Variable: time
Group: biota, Variable: area
Group: biota, Variable: bio_group
Group: biota, Variable: species
Group: biota, Variable: sr90_dl
Group: biota, Variable: sr90_unit
Group: biota, Variable: cs137_dl
Group: biota, Variable: cs137_unit
Group: biota, Variable: pu238_dl
Group: biota, Variable: pu238_unit
Group: biota, Variable: pu239_240_tot_dl
Group: biota, Variable: pu239_240_tot_unit
Group: seawater, Variable: sample
Group: seawater, Variable: lon
Group: seawater, Variable: lat
Group: seawater, Variable: depth
Group: seawater, Variable: time
Group: seawater, Variable: area
Group: seawater, Variable: sr90_dl
Group: seawater, Variable: sr90_unit
Group: seawater, Variable: cs137_dl
Group: seawater, Variable: cs137_unit
Group: seawater, Variable: pu239_240_tot_dl
Group: seawater, Variable: pu239_240_tot_unit
Group: sediment, Variable: sample
Grou

In [ ]:
encode(fname_in, dir_dest, nc_tpl_path(), verbose=False)

Encoding ...


  0%|          | 0/460 [00:00<?, ?it/s]

402-CCHDO-2018.nc


  0%|          | 1/460 [00:03<26:57,  3.52s/it]

374-Ístlund-et-al-1987.nc


...

380-Smith-et-al-2020.nc


  5%|▌         | 23/460 [00:56<13:09,  1.81s/it]

432-Efurd-et-al-1997.nc


  5%|▌         | 23/460 [00:57<18:15,  2.51s/it]


KeyboardInterrupt: 